In [15]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.measure import label
from skimage.io import imread
from skimage.measure import regionprops
from skimage.segmentation import expand_labels
from scipy.special import softmax

In [11]:
methods = ["Mesmer", "Stardist", "Cellpose", "UnMicst"]
datapath = "/home/groups/ChangLab/dataset/HMS-TMA-TNP/OHSU-TMA/Segmentations/"
sample_ids = ['Scene 002','Scene 003', 'Scene 017', 'Scene 049', 'Scene 059']

# Create Probabilty Masks with 12 Radius

The first step here is to generate the 'probability masks' from the original segmentation masks for every sample for every method. This is done by first zeroing out the mask, then taking the point at the center of each segmented cell, setting it equal to 1, and dilating out with a disc shape and a 12-pixel radius (the radius length varies).

In [12]:
def get_mask(datapath, sid, method):
    """load raw segmentation mask for method/sid"""
    datapath = os.path.join(datapath,method)
    if method == 'Mesmer': datapath = os.path.join(datapath, 'mesmer_nuc')
    if method == 'Stardist': datapath = os.path.join(datapath, 'vf')
    files = os.listdir(datapath)
    sid_file = [f for f in files if sid in f][0]
    mask_path = os.path.join(datapath, sid_file)
    print(mask_path)
    return imread(mask_path)

In [13]:
def create_pmask(seg_mask, r):
    """converts a standard segmentation mask into one with a dilated area around 
    the center of the original mask"""
    mask = np.zeros_like(seg_mask)
    rps = regionprops(seg_mask)
    
    #get center of each cell and set the value to one
    centroids = np.array(
        list(map(lambda x : np.array(x.centroid).astype(int), rps))
    )
    mask[centroids[:,0], centroids[:,1]] = 1
    
    #dilate around the center of the cell with the radius size and binarize the mask
    #mask = binary_dilation(mask, disk(r))
    mask = expand_labels(label(mask), distance=r)
    mask = (mask > 0).astype('int')
    
    return mask

In [33]:
pmasks = {}

for sid in sample_ids:
    print()
    print(f'generating pmasks for {sid=}')
    pmasks[sid] = {}
    
    for method in methods:
        print(f'generating pmask for {method=}')
        pmasks[sid][method] = create_pmask(get_mask(datapath, sid, method), 12)


generating pmasks for sid='Scene 002'
generating pmask for method='Mesmer'
/home/groups/ChangLab/dataset/HMS-TMA-TNP/OHSU-TMA/Segmentations/Mesmer/mesmer_nuc/BR1506-A015 - Scene 002_both_nuc.tif
generating pmask for method='Stardist'
/home/groups/ChangLab/dataset/HMS-TMA-TNP/OHSU-TMA/Segmentations/Stardist/vf/BR1506-A015 - Scene 002.png
generating pmask for method='Cellpose'
/home/groups/ChangLab/dataset/HMS-TMA-TNP/OHSU-TMA/Segmentations/Cellpose/BR1506-A015 - Scene 002.png_mask.png
generating pmask for method='UnMicst'
/home/groups/ChangLab/dataset/HMS-TMA-TNP/OHSU-TMA/Segmentations/UnMicst/BR1506-A015 - Scene 002.tif

generating pmasks for sid='Scene 003'
generating pmask for method='Mesmer'
/home/groups/ChangLab/dataset/HMS-TMA-TNP/OHSU-TMA/Segmentations/Mesmer/mesmer_nuc/BR1506-A015 - Scene 003_both_nuc.tif
generating pmask for method='Stardist'
/home/groups/ChangLab/dataset/HMS-TMA-TNP/OHSU-TMA/Segmentations/Stardist/vf/BR1506-A015 - Scene 003.png
generating pmask for method='Ce

#  Create Pseudo-Ground-Truths

In [91]:
weights = np.ones((len(sample_ids), len(methods) - 1)) * (1/ (len(methods) - 1))
agree_ratio = 2/3

In [92]:
def get_pseudo_ground_truths(pmasks, sample_ids, methods, weights, agree_ratio, filtered=False):
    pseudo_gts = {}

    for s,sid in enumerate(sample_ids):
        print()
        print(f'Generating PGTs for SID:{sid}')
        pseudo_gts[sid] = {}
        
        for method_left_out in methods:
            print(f'Generating PGT with {method_left_out} left out')
            if filtered:
                pmasks_ = pmasks[method_left_out][sid] #filtered with resepect to method-left-out PGT 
                print(pmasks_.keys())
            else:
                pmasks_ = pmasks[sid] #unfiltered pmasks are the same for each method/sid, regardless of PGT
        
            methods_in = [method for method in methods if method != method_left_out]
            print(f'using {methods_in}')
            print()
            pgt = sum([pmasks_[method] * weights[s][m] for m,method in enumerate(methods_in)])
            pseudo_gts[sid][method_left_out] = (pgt >= agree_ratio).astype('int')
            
    return pseudo_gts

In [93]:
pseudo_gts = get_pseudo_ground_truths(pmasks=pmasks, 
                                      sample_ids=sample_ids, 
                                      methods=methods, 
                                      weights=weights, 
                                      agree_ratio=agree_ratio,
                                      filtered=False)


Generating PGTs for SID:Scene 002
Generating PGT with Mesmer left out
using ['Stardist', 'Cellpose', 'UnMicst']

Generating PGT with Stardist left out
using ['Mesmer', 'Cellpose', 'UnMicst']

Generating PGT with Cellpose left out
using ['Mesmer', 'Stardist', 'UnMicst']

Generating PGT with UnMicst left out
using ['Mesmer', 'Stardist', 'Cellpose']


Generating PGTs for SID:Scene 003
Generating PGT with Mesmer left out
using ['Stardist', 'Cellpose', 'UnMicst']

Generating PGT with Stardist left out
using ['Mesmer', 'Cellpose', 'UnMicst']

Generating PGT with Cellpose left out
using ['Mesmer', 'Stardist', 'UnMicst']

Generating PGT with UnMicst left out
using ['Mesmer', 'Stardist', 'Cellpose']


Generating PGTs for SID:Scene 017
Generating PGT with Mesmer left out
using ['Stardist', 'Cellpose', 'UnMicst']

Generating PGT with Stardist left out
using ['Mesmer', 'Cellpose', 'UnMicst']

Generating PGT with Cellpose left out
using ['Mesmer', 'Stardist', 'UnMicst']

Generating PGT with UnMics

# Filter Probability Masks based off generated PGTs

In [94]:
def filter_pmask(mask, pgt):
    """filter probability mask for a single method (mask) using the thresholded, averaged, proability mask
    such that the new probability mask contains cell regions that only overlap with one cell region in the
    pseudo-ground-truth"""
    #copy 
    filtered = mask.copy()
    mask_lab = label(mask)
    rps = regionprops(mask_lab)
    
    pgt = label(pgt)
    
    for rp in rps: #for each cell region
        #get cell center coordinates from single-method mask
        coords = rp.coords
        #use those coords to get values of averaged mask 
        vals = pgt[coords[:,0], coords[:,1]]
        uniq, counts = np.unique(vals, return_counts=True)
        
        #ignore background
        if uniq[0] == 0:
            uniq = uniq[1:]
            counts = counts[1:]
            
        n_unique = len(uniq)
        
        #if more than 1 value, zero out pixels in mask that do not equal the most common value
        if n_unique > 1:
            amax = np.argmax(counts)
            top_val = uniq[amax]
            idxs = np.where(vals != top_val)
            to_zero = coords[idxs,:][0]
            filtered[to_zero[:,0], to_zero[:,1]] = False
            
    return filtered

filter probability masks so that each cell region in the probability mask can only vote for one cell label in the pseudo-ground truth mask. 

In [95]:
f_pmasks = {}

for method_out in methods:
    f_pmasks[method_out] = {}
    
    for sid in sample_ids:
        f_pmasks[method_out][sid] = {}

        for method_in in methods:
            if method_in == method_out: continue 

            f_pmasks[method_out][sid][method_in] = filter_pmask(mask=pmasks[sid][method_in], 
                                                                pgt=pseudo_gts[sid][method_out])

# Regenerate PGTs with filtered pmasks

Now we need to regenerate these pseudo-ground-truth masks, this time using filtered probability masks where each probability mask is filtered down to where one pseudo-cell-label in the  probability mask can only vote for one cell label in the pseudo-ground truth mask. 

In [96]:
f_pseudo_gts =  get_pseudo_ground_truths(pmasks=f_pmasks, 
                                         sample_ids=sample_ids,
                                         methods=methods, 
                                         weights=weights, 
                                         agree_ratio=agree_ratio, 
                                         filtered=True)


Generating PGTs for SID:Scene 002
Generating PGT with Mesmer left out
dict_keys(['Stardist', 'Cellpose', 'UnMicst'])
using ['Stardist', 'Cellpose', 'UnMicst']

Generating PGT with Stardist left out
dict_keys(['Mesmer', 'Cellpose', 'UnMicst'])
using ['Mesmer', 'Cellpose', 'UnMicst']

Generating PGT with Cellpose left out
dict_keys(['Mesmer', 'Stardist', 'UnMicst'])
using ['Mesmer', 'Stardist', 'UnMicst']

Generating PGT with UnMicst left out
dict_keys(['Mesmer', 'Stardist', 'Cellpose'])
using ['Mesmer', 'Stardist', 'Cellpose']


Generating PGTs for SID:Scene 003
Generating PGT with Mesmer left out
dict_keys(['Stardist', 'Cellpose', 'UnMicst'])
using ['Stardist', 'Cellpose', 'UnMicst']

Generating PGT with Stardist left out
dict_keys(['Mesmer', 'Cellpose', 'UnMicst'])
using ['Mesmer', 'Cellpose', 'UnMicst']

Generating PGT with Cellpose left out
dict_keys(['Mesmer', 'Stardist', 'UnMicst'])
using ['Mesmer', 'Stardist', 'UnMicst']

Generating PGT with UnMicst left out
dict_keys(['Mesmer',

# Generate F1 table

Now we need to generate f1 table

In [97]:
def f1_score(gt, method):
    """returns precision and recall for a pair of masks"""
    
    ### PRECISION ###
    m_labs = label(method)
    rps = regionprops(m_labs)
    coords = list(map(lambda x : x.coords, rps))
    correct = 0
    print(f'number of regions: {len(rps)}')
    
    for c in coords:
        correct += (gt[c[:,0], c[:,1]]).max()
    print(f'num correct for precision: {correct}')
    precision = correct / len(rps)
    
    ### RECALL ###
    gt_labs = label(gt)
    gt_rps = regionprops(gt_labs)
    coords = list(map(lambda x : x.coords, gt_rps))
    correct = 0
    print(f'number of gt regions: {len(gt_rps)}')
    
    for c in coords:
        correct += (method[c[:,0], c[:,1]]).max() > 0
    print(f'num correct for recall: {correct}')
    recall = correct / len(gt_rps)
    
    assert precision <= 1
    assert precision >= 0
    assert recall <= 1
    assert recall >= 0
    
    f1 = lambda p,r: 2 * (p * r) / (p + r)
    #f1 = lambda p,r: (p + r)/2
    return precision, recall, f1(precision, recall)

In [98]:
f1_table = np.zeros((len(sample_ids), len(methods), len(methods)))
p_table = np.zeros((len(sample_ids), len(methods), len(methods)))
r_table = np.zeros((len(sample_ids), len(methods), len(methods)))

for s,sid in enumerate(sample_ids):
    for i,method_to_eval in enumerate(methods):
        for j,method_left_out in enumerate(methods):
            if method_left_out == method_to_eval: continue
            print(f'{method_to_eval=} {method_left_out=}')
            p_table[s][i][j],r_table[s][i][j],f1_table[s][i][j] = f1_score(gt=f_pseudo_gts[sid][method_left_out], 
                                                                           method=f_pmasks[method_left_out][sid][method_to_eval])
            print()

method_to_eval='Mesmer' method_left_out='Stardist'
number of regions: 10578
num correct for precision: 10264
number of gt regions: 9687
num correct for recall: 9628

method_to_eval='Mesmer' method_left_out='Cellpose'
number of regions: 10575
num correct for precision: 10179
number of gt regions: 9428
num correct for recall: 9369

method_to_eval='Mesmer' method_left_out='UnMicst'
number of regions: 10584
num correct for precision: 10377
number of gt regions: 9946
num correct for recall: 9705

method_to_eval='Stardist' method_left_out='Mesmer'
number of regions: 9345
num correct for precision: 9068
number of gt regions: 8919
num correct for recall: 8534

method_to_eval='Stardist' method_left_out='Cellpose'
number of regions: 9337
num correct for precision: 9133
number of gt regions: 9428
num correct for recall: 8974

method_to_eval='Stardist' method_left_out='UnMicst'
number of regions: 9340
num correct for precision: 9173
number of gt regions: 9946
num correct for recall: 8691

method_t

number of regions: 4139
num correct for precision: 4095
number of gt regions: 4047
num correct for recall: 3915

method_to_eval='Stardist' method_left_out='Mesmer'
number of regions: 3841
num correct for precision: 3696
number of gt regions: 3800
num correct for recall: 3580

method_to_eval='Stardist' method_left_out='Cellpose'
number of regions: 3842
num correct for precision: 3645
number of gt regions: 3770
num correct for recall: 3543

method_to_eval='Stardist' method_left_out='UnMicst'
number of regions: 3845
num correct for precision: 3743
number of gt regions: 4047
num correct for recall: 3639

method_to_eval='Cellpose' method_left_out='Mesmer'
number of regions: 4084
num correct for precision: 3985
number of gt regions: 3800
num correct for recall: 3759

method_to_eval='Cellpose' method_left_out='Stardist'
number of regions: 4085
num correct for precision: 3957
number of gt regions: 3910
num correct for recall: 3830

method_to_eval='Cellpose' method_left_out='UnMicst'
number of 

We assign a weight to a method by quantifying that method's impact on the overall agreeance between the pseudo-ground-truth (PGT) and the other methods. For example, in order to find out how the presence of Stardist in the PGT label effects the correspondence between the PGT and the Mesmer segmentation, we first calculate the F1 scores between the Mesmer segmentation and each of the PGTs comprised of Mesmer and two of the three other methods i.e. PGT({Mesmer,Cellpose,UnMicst}), PGT({Mesmer, Stardist, UnMicst}), and PGT({Mesmer, Stardist, Cellpose}). If the F1 score between Mesmer and the PGT with Stardist left out i.e. PGT({Mesmer,Cellpose,UnMicst}) is high relative to the F1 scores between Mesmer and the other two PGTs containing Stardist, that indicates that Stardist is a bad contributor to the PGT, with respect to Mesmer. If the same holds true for Cellpose, and UnMicst, then it follows that the contribution of Stardist to the PGT should be discounted by assigning a lower weight to its vote.

In [99]:
column_names = [f'PGT({[m for m in methods if m != m_out]})' for m_out in methods]

precision = 'How many predictions are in the ground-truth?'
recall = 'how many ground-truths are in the prediction?'

In [100]:
#with unfiltered pmask
#pd.DataFrame(f1_table[0], index=methods, columns=[f'{m}-left-out' for m in methods])
pd.DataFrame(f1_table[0], index=methods, columns=column_names)

,"PGT(['Stardist', 'Cellpose', 'UnMicst'])","PGT(['Mesmer', 'Cellpose', 'UnMicst'])","PGT(['Mesmer', 'Stardist', 'UnMicst'])","PGT(['Mesmer', 'Stardist', 'Cellpose'])"
Mesmer,0.000000,0.981971,0.977899,0.978100
Stardist,0.963549,0.000000,0.964819,0.924809
Cellpose,0.981266,0.983971,0.000000,0.963747
UnMicst,0.760111,0.725016,0.741538,0.000000


In [101]:
### with unfiltered pmask
#pd.DataFrame(p_table[0], index=methods, columns=[f'{m}-left-out' for m in methods])
pd.DataFrame(p_table[0], index=methods, columns=column_names)

,"PGT(['Stardist', 'Cellpose', 'UnMicst'])","PGT(['Mesmer', 'Cellpose', 'UnMicst'])","PGT(['Mesmer', 'Stardist', 'UnMicst'])","PGT(['Mesmer', 'Stardist', 'Cellpose'])"
Mesmer,0.000000,0.970316,0.962553,0.980442
Stardist,0.970358,0.000000,0.978151,0.982120
Cellpose,0.976153,0.989792,0.000000,0.995629
UnMicst,0.994902,0.995401,0.996468,0.000000


In [102]:
#with unfiltered pmask
#pd.DataFrame(r_table[0], index=methods, columns=[f'{m}-left-out' for m in methods])
pd.DataFrame(r_table[0], index=methods, columns=column_names)

,"PGT(['Stardist', 'Cellpose', 'UnMicst'])","PGT(['Mesmer', 'Cellpose', 'UnMicst'])","PGT(['Mesmer', 'Stardist', 'UnMicst'])","PGT(['Mesmer', 'Stardist', 'Cellpose'])"
Mesmer,0.000000,0.993909,0.993742,0.975769
Stardist,0.956834,0.000000,0.951846,0.873819
Cellpose,0.986433,0.978218,0.000000,0.933843
UnMicst,0.614979,0.570146,0.590475,0.000000


Mesmer, Stardist, and Cellpose have a higher f1 on average with the PGT({Mesmer, Stardist, Cellpose}) relative to PGT({Mesmer, Stardist, UnMicst}), PGT({Mesmer, Cellpose, UnMicst}), and PGT({Stardist, Cellpose, UnMicst}) implying that UnMicst displays poor contribution, when the agreeance ratio is 3/3.

However, when the agreeance ratio is 2/3, then Mesmer, Stardist, and Cellpose have a lower f1 on average with the PGT({Mesmer, Stardist, Cellpose}), implying that UnMicst displays poor contribution.

Now we need to convert to a delta table representing the relative change in performance caused by dropping a single method from the ensemble used for creating the PGT. We obtain this delta by averaging the non-zero F1 scores in each row, and subtracting that mean from each non-zero value in the row and dividing by the average, which gives us a normalized delta value. Finally, we multiply each value by 100 to obtain a percentage.

In [111]:
softmax([0.0116, 0.0167, 0.0192, -0.0475]).round(2)

array([0.25, 0.25, 0.25, 0.24])

In [138]:
sample_ids[0]

'Scene 002'

In [142]:
f1_table[0]

array([[       nan, 0.98197086, 0.977899  , 0.97810008],
       [0.96354865,        nan, 0.96481923, 0.92480936],
       [0.98126645, 0.98397092,        nan, 0.96374665],
       [0.7601107 , 0.7250163 , 0.74153827,        nan]])

In [145]:
f1_t = f1_table[0]
f1_t[f1_t == 0] = np.nan
mF1 = np.nanmean(f1_t, axis=1)
print(mF1)
mean_mF1 = np.mean(mF1)
z = (mF1 - mean_mF1) / mean_mF1
print(z)
print(softmax(z))

[0.97932331 0.95105908 0.97632801 0.74222176]
[ 0.0735451   0.04256154  0.07026161 -0.18636826]
[0.26757057 0.2594074  0.26669345 0.20632859]


In [136]:
f1_t = f1_table[0]
f1_delta_table = np.zeros(f1_t.shape)
f1_t[f1_t==0] = np.nan
for m,method in enumerate(methods):
    mean_col = np.nanmean(f1_t[:,m])
    print(mean_col)
    for i,f1 in enumerate(f1_t[:,m]):
        if np.isnan(f1):
            f1_delta_table[i,m] = np.nan
        else:   
            f1_delta_table[i,m] = 100 * (f1 - mean_col) / mean_col

0.9016419318063336
0.8969860262529866
0.8947521665203345
0.9555520329117567


In [134]:
f1_delta_table

array([[        nan, -9.47448771, -9.29272214, -2.35968843],
       [-6.86599766,         nan, -7.83089049,  3.21726783],
       [-8.83105724, -9.69746344,         nan, -0.8575794 ],
       [15.6970549 , 19.17195115, 17.12361262,         nan]])

In [112]:
weights  = np.zeros((len(sample_ids),len(methods))) 
for s,sid in enumerate(sample_ids):
    f1_t = f1_table[s]
    f1_t[f1_t == 0] = np.nan
    mF1 = np.nanmean(f1_t, axis=1)
    mean_mF1 = np.mean(mF1)
    z = (mF1 - mean_mF1) / mean_mF1
    print(z)
    weights[s] = softmax(z)

[ 0.0735451   0.04256154  0.07026161 -0.18636826]
[ 0.01444228  0.02668536  0.04534674 -0.08647438]
[ 3.88435097e-02 -3.01122100e-05  3.11952143e-02 -7.00086118e-02]
[ 0.03092616  0.00954085  0.03208051 -0.07254752]
[ 0.04627907  0.00539881  0.04374742 -0.0954253 ]


In [29]:
f1_delta_table = np.zeros((len(sample_ids), len(methods), len(methods)))

for s,_ in enumerate(f1_table):#get table for each sample
    for i,f1_i in enumerate(f1_table[s]):#get row for each method
        mean_f1 = np.mean([f1 for f1 in f1_i if f1 != 0])

        for j,f1 in enumerate(f1_i):
            if i == j: continue
            f1_delta_table[s][i][j] = 100 * (f1 - mean_f1) / mean_f1

In [30]:
pd.DataFrame(f1_delta_table[0], index=methods, columns=[f'{m} dropped' for m in methods])

,Mesmer dropped,Stardist dropped,Cellpose dropped,UnMicst dropped
Mesmer,0.000000,0.270344,-0.145439,-0.124906
Stardist,1.313227,0.000000,1.446824,-2.760051
Cellpose,0.505818,0.782822,0.000000,-1.288640
UnMicst,2.410189,-2.318102,-0.092087,0.000000


Finally, calculate weights

In [31]:
weights  = np.zeros((len(sample_ids),len(methods)))
avg_deltas  = np.zeros((len(sample_ids),len(methods)))

for s,sid in enumerate(sample_ids):
    delta_table = f1_delta_table[s]
    for m, method in enumerate(methods):
        avg_deltas[s][m] = -np.mean([f1_d for f1_d in delta_table[:,m] if f1_d != 0])

    weights[s] = softmax(avg_deltas[s])

In [32]:
weights

array([[0.03782273, 0.23611048, 0.10349831, 0.62256848],
       [0.16076133, 0.20273457, 0.20088619, 0.43561791],
       [0.08858254, 0.28050836, 0.22673281, 0.40417629],
       [0.21646338, 0.26615496, 0.2739616 , 0.24342005],
       [0.14993142, 0.36212145, 0.1616214 , 0.32632573]])